In [1]:
from mip import Model, xsum, maximize, BINARY

p = [10, 13, 18, 31, 7, 15]
w = [11, 15, 20, 35, 10, 33]
c, I = 47, range(len(w))

m = Model("knapsack")

x = [m.add_var(var_type=BINARY) for i in I]

m.objective = maximize(xsum(p[i] * x[i] for i in I))

m += xsum(w[i] * x[i] for i in I) <= c

m.optimize()

selected = [i for i in I if x[i].x >= 0.99]
print("selected items: {}".format(selected))

selected items: [0, 3]


Simple Time-Series Example with Binary Decision Variables

In [2]:
from mip import Model, xsum, maximize, BINARY

# Example time series data
revenues = [100, 150, 200, 250, 300]
costs = [50, 60, 70, 80, 90]
budgets = [200, 250, 300, 350, 400]

# Create a new MIP model
m = Model("revenue_optimization")

# Define decision variables
T = len(revenues)  # number of time periods
x = [m.add_var(var_type=BINARY) for t in range(T)]

# Set objective function to maximize revenue
m.objective = maximize(xsum(revenues[t] * x[t] - costs[t] * x[t] for t in range(T)))

# Add budget constraint for each time period
for t in range(T):
    m += xsum(costs[i] * x[i] for i in range(t+1)) <= budgets[t]

# Solve the optimization problem
m.optimize()

# Print the solution
print("Revenue: ${}".format(m.objective_value))
print("Selected time periods: {}".format([t+1 for t in range(T) if x[t].x >= 0.99]))


Revenue: $650.0
Selected time periods: [1, 2, 3, 4, 5]


Simple Time-Series Example with Non-Binary Decision Variables

In [9]:
from itertools import product
from sys import stdout as out
from mip import Model, xsum, minimize, BINARY

# names of places to visit
places = ['Antwerp', 'Bruges', 'C-Mine', 'Dinant', 'Ghent',
          'Grand-Place de Bruxelles', 'Hasselt', 'Leuven',
          'Mechelen', 'Mons', 'Montagne de Bueren', 'Namur',
          'Remouchamps', 'Waterloo']

# distances in an upper triangular matrix
dists = [[83, 81, 113, 52, 42, 73, 44, 23, 91, 105, 90, 124, 57],
         [161, 160, 39, 89, 151, 110, 90, 99, 177, 143, 193, 100],
         [90, 125, 82, 13, 57, 71, 123, 38, 72, 59, 82],
         [123, 77, 81, 71, 91, 72, 64, 24, 62, 63],
         [51, 114, 72, 54, 69, 139, 105, 155, 62],
         [70, 25, 22, 52, 90, 56, 105, 16],
         [45, 61, 111, 36, 61, 57, 70],
         [23, 71, 67, 48, 85, 29],
         [74, 89, 69, 107, 36],
         [117, 65, 125, 43],
         [54, 22, 84],
         [60, 44],
         [97],
         []]

# number of nodes and list of vertices
n, V = len(dists), set(range(len(dists)))

# distances matrix
c = [[0 if i == j
      else dists[i][j-i-1] if j > i
      else dists[j][i-j-1]
      for j in V] for i in V]

model = Model()

# binary variables indicating if arc (i,j) is used on the route or not
x = [[model.add_var(var_type=BINARY) for j in V] for i in V]

# continuous variable to prevent subtours: each city will have a
# different sequential id in the planned route except the first one
y = [model.add_var() for i in V]

# objective function: minimize the distance
model.objective = minimize(xsum(c[i][j]*x[i][j] for i in V for j in V))

# constraint : leave each city only once
for i in V:
    model += xsum(x[i][j] for j in V - {i}) == 1

# constraint : enter each city only once
for i in V:
    model += xsum(x[j][i] for j in V - {i}) == 1

# subtour elimination
for (i, j) in product(V - {0}, V - {0}):
    if i != j:
        model += y[i] - (n+1)*x[i][j] >= y[j]-n

# optimizing
model.optimize()

# checking if a solution was found
if model.num_solutions:
    out.write('route with total distance %g found: %s'
              % (model.objective_value, places[0]))
    nc = 0
    while True:
        nc = [i for i in V if x[nc][i].x >= 0.99][0]
        out.write(' -> %s' % places[nc])
        if nc == 0:
            break
    out.write('\n')

route with total distance 547 found: Antwerp -> Mechelen -> Leuven -> Hasselt -> C-Mine -> Montagne de Bueren -> Remouchamps -> Dinant -> Namur -> Mons -> Waterloo -> Grand-Place de Bruxelles -> Ghent -> Bruges -> Antwerp


In [2]:
from mip import Model, xsum, maximize, BINARY

# Define parameters
capacity = 10000  # Battery capacity in MWh
wind_energy = [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]  # Wind energy in MW
day_prices = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 90, 80, 70, 60, 50, 40, 30, 20, 10, 5, 5, 5, 5, 5]  # Day-ahead prices in EUR/MWh
bal_prices = [20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 100, 90, 80, 70, 60, 50, 40, 30, 20, 10, 5, 5, 5, 5]  # Balancing prices in EUR/MWh

# Create model
model = Model()

# Define decision variables
E_day = [model.add_var() for t in range(24)]

# Define objective function
obj = xsum(day_prices[t] * E_day[t] for t in range(24))
model.objective = maximize(obj)

# Add capacity constraints
for t in range(24):
    model += xsum(wind_energy[t] - E_day[t] for t in range(24)) == 0

for t in range(24):
    model += (xsum(E_day[t] for t in range(24)) <= 100)

# for t in range(24):   
    # model += xsum(E_bal[t] for t in range(24)) <= 100

# Add operational constraints
# model.add_constr(xsum(wind_energy[t] - E_day[t] - E_bal[t] for t in range(24)) == capacity)
# model.add_constr(xsum(E_day[t] + E_bal[t] - wind_energy[t] for t in range(24)) == capacity)

# Solve model
model.optimize()

# Print solution
if model.status == 1:
    print(f"Optimal solution found: revenue = {model.objective_value:.2f} EUR")
    for t in range(24):
        print(f"Hour {t}: E_day = {E_day[t].x:.2f} MWh, E_bal = {E_bal[t].x:.2f} MWh")
else:
    print("No optimal solution found.")


from mip import Model, xsum, maximize, BINARY

# Example time series data
revenues = [100, 150, 200, 250, 300]
costs = [50, 60, 70, 80, 90]
budgets = [200, 250, 300, 350, 400]

# Create a new MIP model
m = Model("revenue_optimization")

# Define decision variables
T = len(revenues)  # number of time periods
x = [m.add_var(var_type=BINARY) for t in range(T)]

# Set objective function to maximize revenue
m.objective = maximize(xsum(revenues[t] * x[t] - costs[t] * x[t] for t in range(T)))

# Add budget constraint for each time period
for t in range(T):
    m += xsum(costs[i] * x[i] for i in range(t+1)) <= budgets[t]

# Solve the optimization problem
m.optimize()

# Print the solution
print("Revenue: ${}".format(m.objective_value))
print("Selected time periods: {}".format([t+1 for t in range(T) if x[t].x >= 0.99]))



No optimal solution found.


In [4]:
# Import the MIP library
from mip import Model, xsum, maximize, BINARY

# Define the parameters
profit = [10, 15]  # Profit per unit for A and B, respectively
labor_req = [2, 3]  # Labor requirement per unit for A and B, respectively
raw_mat_req = [1, 2]  # Raw material requirement per unit for A and B, respectively
total_labor = 100  # Total labor available
total_raw_mat = 80  # Total raw materials available
demand = [20, 30]  # Minimum demand for A and B, respectively

# Create the model
m = Model()

# Define the decision variables
x = [m.add_var(var_type=BINARY) for i in range(2)]  # x[0] = number of units of A to produce, x[1] = number of units of B to produce

# Define the objective function (maximize profit)
m.objective = maximize(xsum(profit[i] * x[i] for i in range(2)))

# Add the constraints
m += xsum(labor_req[i] * x[i] for i in range(2)) <= total_labor  # Labor constraint
m += xsum(raw_mat_req[i] * x[i] for i in range(2)) <= total_raw_mat  # Raw material constraint
m += x[0] >= demand[0]  # Demand constraint for A
m += x[1] >= demand[1]  # Demand constraint for B

# Solve the model
m.optimize()

# Print the solution
print("Number of units of A to produce: ", x[0].x)
print("Number of units of B to produce: ", x[1].x)
print("Maximum profit: $", m.objective_value)


Number of units of A to produce:  None
Number of units of B to produce:  None
Maximum profit: $ None
